<a href="https://colab.research.google.com/github/mingyungkim/SarcasmDetection/blob/master/SarcasmDetection_MainCodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone Git

Run the following to get the required files needed for this assignment. 

TODO: switch repos

In [1]:
!gdown https://drive.google.com/uc?id=1-0FWzpJLIAROL36EUgX8upfYK6nKUls9
!git clone https://github.com/cis700/hw3-solutions.git
!mv hw3-solutions/* .
!rm -rf hw3-solutions/
# !wget "https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/extract_features.py"
!wget "https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/lm_finetuning/finetune_on_pregenerated.py"
!wget "https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/lm_finetuning/pregenerate_training_data.py"

Downloading...
From: https://drive.google.com/uc?id=1-0FWzpJLIAROL36EUgX8upfYK6nKUls9
To: /content/pytorch_model.bin
220MB [00:01, 170MB/s]
Cloning into 'hw3-solutions'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 2), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (17/17), done.
--2019-05-15 14:53:32--  https://raw.githubusercontent.com/huggingface/pytorch-pretrained-BERT/master/examples/lm_finetuning/finetune_on_pregenerated.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16201 (16K) [text/plain]
Saving to: ‘finetune_on_pregenerated.py’

finetune_on_pregene 100%[===================>]  15.82K  --.-KB/s    in 0.01s   

2019-05-15 14:53

# Part 1.  Setting up the Reddit dataset

For this assignment, we are going to use the Reddit sarcasm dataset.  Since sarcasm is difficult to express via text, Redditors frequently end sarcastic comments with "/s".  The Reddit dataset uses the presence of this "/s" token to create a labeled dataset of sarcastic and non-sarcastic comments.  The original Reddit dataset can be found here: https://www.kaggle.com/danofer/sarcasm

We've made some slight modifications to the dataset -- we've removed metadata and balanced the dataset (only 1% of Reddit comments are actually sarcastic, but 50% of the training and test examples are sarcastic in the modified dataset).

Note that even within the niche of sarcasm-based NLP, there are better, cleaner, and larger datasets than this Reddit dataset.  I've selected this one specifically because there are many teachable characteristics of the dataset.

Run the following commands to unzip the csv file.

In [0]:
!gzip -d ./data/reddit_train.csv.gz
!gzip -d ./data/reddit_test.csv.gz

The following cells have all of the necessary imports for this homework.  We see 2 new packages.


1.   **torchtext**.  Recall that torchvision built many image datasets, CV models, and image processing utilities into the PyTorch framework.  The torchtext package does the same for NLP -- it has many utilities for handling tokenization, variable-length sequences, feature vectorization.
2.   **spaCy**.  This is a state-of-the-art English language tokenizer.  We will pass this as an input to the torchtext equivalent of a DataLoader.





In [3]:
# !pip install spacy
# !python -m spacy download en
!pip install bert-pytorch #ADDED BY MK
!pip install pytorch-pretrained-bert #ADDED BY MK

     |████████████████████████████████| 133kB 2.8MB/s 


In [0]:
import torch
import torchtext
import torchtext.data as data
from torchtext.vocab import Vectors
import gc
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import random
import os
import csv
import time  # TODO: remove
from google.colab import drive
from helper import Logger
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler,RandomSampler
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM #ADDED BY MK
from pytorch_pretrained_bert import BertConfig, BertForTokenClassification, BertAdam #ADDED BY MK

In [5]:
# reset all environment conditions

def reset_env():
    SEED = 1234
    random.seed(SEED)
    torch.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

drive.mount('/content/gdrive')
os.makedirs('/content/gdrive/My Drive/hw-3-models', exist_ok=True)
os.makedirs('/content/gdrive/My Drive/hw-3-models/training', exist_ok=True)
os.makedirs('/content/gdrive/My Drive/hw-3-models/finetuned_lm', exist_ok=True)
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
cuda:0


In [0]:
def read_write_examples_for_fine_tune(input_file):
    """Read a list of `InputExample`s from an input file."""
    with open(input_file, "r", encoding='utf-8') as reader:
      with open("my_corpus.txt",'w+') as out_file:
        csv_reader = csv.reader(reader, delimiter=',')
        for line in csv_reader:
            if line[0]=='':
              continue
            label=int(line[1])
            comment = line[2]
            parent = line[3]
            out_file.write(parent+'\n')
            out_file.write('\n')
            out_file.write(comment+'\n')
            out_file.write('\n')
    return
  
# read_write_examples_for_fine_tune('./data/reddit_train.csv')

In [0]:
# !python3 pregenerate_training_data.py --train_corpus my_corpus.txt --bert_model bert-base-uncased --do_lower_case --output_dir "/content/gdrive/My Drive/hw-3-models/training/" --epochs_to_generate 1 --max_seq_len 128

In [0]:
# !git clone https://github.com/NVIDIA/apex
# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex/

In [0]:
# !python3 finetune_on_pregenerated.py --pregenerated_data "/content/gdrive/My Drive/hw-3-models/training/" --bert_model bert-base-uncased --do_lower_case --output_dir "/content/gdrive/My Drive/hw-3-models/finetuned_lm/" --epochs 1 --gradient_accumulation_steps 8 --fp16

In [0]:
# raise NotImplementedError




## Q1.1 Review of the dataset
Let's first understand the structure of the dataset.

*   Print out the headers and the first five rows of reddit_train.csv.  Put this in your writeup.
*   As a review of part 1, describe each input and state whether it is fixed-length or variable-length in your writeup.
*   Print out size of both datasets and put the lengths in your writeup.  




In [0]:
# raise NotImplementedError
# reddit_train_df = pd.read_csv('./data/reddit_train.csv')
# reddit_test_df = pd.read_csv('./data/reddit_test.csv')
# print(reddit_train_df.head())

# print(len(reddit_train_df.index))
# print(len(reddit_test_df.index))

## Q1.2 Featurizing the dataset with torchtext

Computer vision has torchvision; NLP has torchtext.  In this problem, we will create and featurize a torchtext dataset with Fields, a data structure that can automatically featurize text with embeddings.

First, create a tokenizer using spacy_en.  Create two torchtext data fields using data.Field from torchtext.data:

*   a sequential field named TEXT for comment and parent_comment.  (*Hint: since this is natural language, this is sequential data.  Use your tokenizer and convert all characters to lowercase.*)
*   a non-sequential field named LABEL for the labels.  (*Hint: since this is a categorical variable, it is not sequential.  Furthermore, it does not require a vocabulary since there are no words to embed.*)

Next, look at the documentation for data.TabularDataset.splits and create `train_ds` and `test_ds`.  You will need the paths to the training and test datasets, the format, and the mapping from columns to fields.  Note that the first column in the csv's should not be a field.  This step creates 3 torchtext objects for each row in the dataset, so it will take some time (between 2-10 minutes).

We then build our vocabulary with GloVE, a word embedding similar to Word2Vec.  Create a vocabulary from TEXT using the train dataset (*Hint: look at the documentation for Field.build_vocab*).  Use the glove.6B.100d word embedding, and save the vocbulary.  The first time you run this, it will take roughly 5-10 minutes to download.  The pretrained model will then be stored in the ./.vector_cache folder, and rerunning this command will take negligible time.  Store the final vocabulary in the `vocab` variable.

Finally, print out a single example from `train_ds` and print out the properties of `vocab` ('freqs', 'itos', 'stoi', 'vectors').  Include this printouts in the writeup.  Explain what the properties of `vocab` are.  These are the only things you need to include in your writeup for Q1.1b.

In [0]:
# spacy_en = spacy.load('en')
# spacy.prefer_gpu()

#def tokenizer(text):  # create a tokenizer function
#    return [tok.text for tok in spacy_en.tokenizer(text)]

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased').tokenize         #ADDED BY MK
  
# TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True)
# LABEL = data.Field(sequential=False, use_vocab=False)

# train_ds, test_ds = data.TabularDataset.splits(
#     path='./data/', train='reddit_train.csv',
#     test='reddit_test.csv', format='csv', skip_header=True,
#     fields=[('index', None), ('label', LABEL), ('comment', TEXT), ('parent_comment', TEXT)])

# TEXT.build_vocab(train_ds)                    # vectors="glove.6B.100d") REVISED BY MK (IS THIS CORRECT?)
# vocab = TEXT.vocab

# print(vocab.__dict__.keys())
# # vocab.freqs
# # vocab.itos
# vocab.stoi
# # vocab.vectors
# raise NotImplemented

In [0]:
# train_ds
# TEXT

In [0]:
class InputExample(object):

    def __init__(self, unique_id, comment, parent,label):
        self.unique_id = unique_id
        self.comment = comment
        self.parent = parent
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, unique_id, input_ids_comment, input_mask_comment, input_type_ids_comment, input_ids_parent, input_mask_parent, input_type_ids_parent,label):
        self.unique_id = unique_id
#         self.tokens_comment = tokens_comment
        self.input_ids_comment = input_ids_comment
        self.input_mask_comment = input_mask_comment
        self.input_type_ids_comment = input_type_ids_comment
#         self.tokens_parent = tokens_parent
        self.input_ids_parent = input_ids_parent
        self.input_mask_parent = input_mask_parent
        self.input_type_ids_parent = input_type_ids_parent
        self.label=label

In [0]:
import csv
def read_examples(input_file):
    """Read a list of `InputExample`s from an input file."""
    examples = []
    unique_id = 0
    maxlen_comment=0
    maxlen_parent=0
    with open(input_file, "r", encoding='utf-8') as reader:
        csv_reader = csv.reader(reader, delimiter=',')
        for line in csv_reader:
            if line[0]=='':
              continue
            label=int(line[1])
            comment = line[2]
            parent = line[3]
            maxlen_comment=max(maxlen_comment,len(comment))
            maxlen_parent=max(maxlen_parent,len(parent))
            examples.append(InputExample(unique_id=unique_id, comment=comment, parent=parent,label=label))
            unique_id += 1
    return examples

In [0]:
def convert_examples_to_features(examples,seq_length,  tokenizer,parent_flag='No_Context'):
    """Loads a data file into a list of `InputFeature`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
#         print(example.comment)
        if parent_flag=="Concatenated":
          comment_tokens = tokenizer.tokenize(example.comment+example.parent)  
        else:
          comment_tokens = tokenizer.tokenize(example.comment)
        if parent_flag=="Separate":
          parent_tokens = tokenizer.tokenize(example.parent)
          if len(parent_tokens) > seq_length-2:
                parent_tokens = parent_tokens[0:(seq_length-2)]

          
        if len(comment_tokens) > seq_length-2:
              comment_tokens = comment_tokens[0:(seq_length-2)]

            
        max_length_comment=seq_length
        max_length_parent=seq_length
        tokens_comment = []
        tokens_comment.append("[CLS]")
        input_type_ids_comment = []
        input_type_ids_comment.append(0)
        for token in comment_tokens:
            tokens_comment.append(token)
            input_type_ids_comment.append(0)
        tokens_comment.append("[SEP]")
#         max_length_comment=max(max_length_comment,len(tokens_comment))
        input_type_ids_comment.append(0)
        input_ids_comment = tokenizer.convert_tokens_to_ids(tokens_comment)          
        input_mask_comment = [1] * len(input_ids_comment)
        # Zero-pad up to the sequence length.
        while len(input_ids_comment) < max_length_comment:
            input_ids_comment.append(0)
            input_mask_comment.append(0)
            input_type_ids_comment.append(0)
        
        
        tokens_parent = []
        input_type_ids_parent = []
        input_mask_parent=[]
        input_ids_parent=[]
        if parent_flag=="Separate":
          tokens_parent.append("[CLS]")
          input_type_ids_parent.append(0)
          for token in parent_tokens:
              tokens_parent.append(token)
              input_type_ids_parent.append(0)
          tokens_parent.append("[SEP]")
#           max_length_parent=max(max_length_parent,len(tokens_parent))
          input_type_ids_parent.append(0)
          input_ids_parent = tokenizer.convert_tokens_to_ids(tokens_parent)
          input_mask_parent = [1] * len(input_ids_parent)
          # Zero-pad up to the sequence length.
          while len(input_ids_parent) < max_length_parent:
              input_ids_parent.append(0)
              input_mask_parent.append(0)
              input_type_ids_parent.append(0)


        features.append(
            InputFeatures(
                unique_id=example.unique_id,
                input_ids_comment=input_ids_comment,
                input_mask_comment=input_mask_comment,
                input_type_ids_comment=input_type_ids_comment,
            
                input_ids_parent=input_ids_parent,
                input_mask_parent=input_mask_parent,
                input_type_ids_parent=input_type_ids_parent,
                label=example.label
            ))
    return features

  
def get_features(file_name,b):
    examples=read_examples(file_name)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    features = convert_examples_to_features(examples=examples,seq_length=128, tokenizer=tokenizer,parent_flag='Separate')
    del examples
    del tokenizer
    gc.collect()
    all_input_ids_comment = torch.tensor([f.input_ids_comment for f in features], dtype=torch.long)
    all_input_mask_comment = torch.tensor([f.input_mask_comment for f in features], dtype=torch.long)
    all_input_ids_parent = torch.tensor([f.input_ids_parent for f in features], dtype=torch.long)
    all_input_mask_parent = torch.tensor([f.input_mask_parent for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids_comment, all_input_mask_comment,all_input_ids_parent, all_input_mask_parent, all_labels)
    eval_sampler = RandomSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=b)
    del features
    del all_input_ids_comment
    del all_input_mask_comment
    del all_input_ids_parent
    del all_input_mask_parent
    del all_labels
    gc.collect()
    return eval_dataloader

## Utility cells for modeling

We've set up a few utility cells here.  Use these as you see fit.

1.   Cell to set up a logger and Tensorboard.
2.   Cell to keep track of hyperparameters.
3.   Cell with functions for training and testing loops.



In [10]:
### Tensorboard setup
# asdf
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

#./ngrok authtoken 7rHP2EU3WwBpFXGh7cH3z_6VS67KiDzaRyVAKTLt8St
    
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

logger = Logger('./logs')

Tensorboard Link: https://4128ea0d.ngrok.io


In [0]:
### hyperparameters
# raise NotImplementedError
# overall
all_models_hyperparameters = {'embedding_dim': 768,                             # revised by MK (100 -> 768)
                              'output_dim': 1,
#                               'vocabulary_size': len(TEXT.vocab),
                              'train_batch_size': 40,
                              'test_batch_size': 1000}

In [0]:
# def process_example(m, ex, variable_length):
#     lab, child, parent = ex.label.to(device), ex.comment.to(device), ex.parent_comment.to(device)
#     if variable_length:
#         lengths = [list(c.size())[0] for c in child.permute(1, 0)]
#         out = torch.squeeze(m(child, parent, torch.LongTensor(lengths).cpu()), 1)
#     else:
#         out = torch.squeeze(m(child, parent), 1)
#     return lab, out


def train_model(model_name, model, optimizer, loss_criterion, num_epochs, parents=False):
    tick = time.time()
    # make sure model and loss are on CUDA
    model = model.to(device)
    loss_criterion = loss_criterion.to(device)

    logger = Logger('./logs/' + model_name + str(time.time()))
    batch_num = 0
    max_accuracy = 0
    for epoch in range(num_epochs):
        print("starting epoch ", epoch, ", ", time.time() - tick)
#         for example in train_iter:
        for feature_input in eval_dataloader:
            if parents==False:
              input_ids= feature_input[0]
              input_mask= feature_input[1]
              label= feature_input[2]
              input_ids = input_ids.to(device)
              input_mask = input_mask.to(device)
              label=label.to(device)
              output=torch.squeeze(model(input_ids,input_mask),1)
            else:
              input_ids_comment= feature_input[0]
              input_mask_comment= feature_input[1]
              input_ids_parent= feature_input[2]
              input_mask_parent= feature_input[3]
              label= feature_input[4]
              input_ids_comment = input_ids_comment.to(device)
              input_mask_comment = input_mask_comment.to(device)
              input_ids_parent = input_ids_parent.to(device)
              input_mask_parent = input_mask_parent.to(device)
              label=label.to(device)
              output=torch.squeeze(model(input_ids_comment,input_mask_comment,input_ids_parent,input_mask_parent),1)
              
              
            batch_num += 1
            
            optimizer.zero_grad()
            loss = loss_criterion(output.float(), label.float())
            loss.backward()
#             print(model.classifier.weight.grad) 
            optimizer.step()

            ## Tensorboard stuff
            
            # computing train accuracy
            predicted = torch.round(torch.sigmoid(output.data))
            print(predicted)
            print(label)
            total = label.size(0)
            correct = (predicted.float() == label.to(device).float()).sum().item()
            accuracy = correct / total
            info = { 'loss': loss, 'accuracy': accuracy }
            print(info)
            
            # computing test accuracy
            if batch_num % 1000 == 0:
                test_total = 0
                test_correct = 0
                with torch.no_grad():
                    batch_count=0
                    for feature_input in test_eval_dataloader:
                        if parents==False:
                          input_ids= feature_input[0]
                          input_mask= feature_input[1]
                          test_label= feature_input[2]
                          input_ids = input_ids.to(device)
                          input_mask = input_mask.to(device)
                          test_label=test_label.to(device)
                          test_output=torch.sigmoid(torch.squeeze(model(input_ids,input_mask),1))
                        else:
                          input_ids_comment= feature_input[0]
                          input_mask_comment= feature_input[1]
                          input_ids_parent= feature_input[2]
                          input_mask_parent= feature_input[3]
                          test_label= feature_input[4]
                          input_ids_comment = input_ids_comment.to(device)
                          input_mask_comment = input_mask_comment.to(device)
                          input_ids_parent = input_ids_parent.to(device)
                          input_mask_parent = input_mask_parent.to(device)
                          test_label=test_label.to(device)
                          test_output=torch.sigmoid(torch.squeeze(model(input_ids_comment,input_mask_comment,input_ids_parent,input_mask_parent),1))


#                     for test_example in test_iter:
#                         test_label, test_output = process_example(model, test_example, variable_length)
                        test_predicted = torch.round(test_output.data)
                        test_total += test_label.size(0)
                        test_correct += (test_predicted.float() == test_label.to(device).float()).sum().item()
                        batch_count=batch_count+1
                        if batch_count>=50:
                            break;  # only takes one test batch
                test_accuracy = test_correct / test_total
                info['test_accuracy'] = test_accuracy                  
                #if test_accuracy > max_accuracy:
                #    torch.save(model.state_dict(), "/content/gdrive/My Drive/hw-3-models/" + model_name + "-" + str(batch_num))

            for tag, value in info.items():
                logger.scalar_summary(tag, value, batch_num + 1)  
            torch.cuda.empty_cache()
    return model

# def test_model_accuracy(model, variable_length=False):
#     model = model.to(device)
#     confusion_mtx = np.zeros((2, 2))
#     test_total = 0
#     test_correct = 0
#     c=0
#     with torch.no_grad():
# #         for test_example in test_iter:
#         for input_ids, input_mask, test_label in eval_dataloader:
#             print(c)
#             c=c+1
#             input_ids = input_ids.to(device)
#             input_mask = input_mask.to(device)
#             test_label=test_label.to(device)
# #             batch_num += 1
# #             label, output = process_example(model, example, variable_length)
#             test_output=torch.sigmoid(torch.squeeze(model(input_ids,input_mask),1))

# #             test_label, test_output = process_example(model, test_example, variable_length)
#             test_predicted = torch.round(test_output.data)
#             test_total += test_label.size(0)
#             test_correct += (test_predicted.float() == test_label.to(device).float()).sum().item()
# #             print(test_correct / test_total)
# #             del test_example
#     test_accuracy = test_correct / test_total
#     return test_accuracy

def test_model_confusion_matrix(model, parents=False):
    model = model.to(device)
    confusion_mtx = np.zeros((2, 2))
    c=0
    with torch.no_grad():
#         for test_example in test_iter:
        for feature_input in test_eval_dataloader:
          if parents==False:
            input_ids= feature_input[0]
            input_mask= feature_input[1]
            test_label= feature_input[2]
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            test_label=test_label.to(device)
            if input_ids.size()[0] > 4:  # ensures we're looking at sufficiently large comments
                test_output=torch.sigmoid(torch.squeeze(model(input_ids,input_mask),1))
                test_predicted = torch.round(test_output.data)
                current_cm = confusion_matrix(test_label.cpu().numpy(), test_predicted.cpu().numpy())
                confusion_mtx += current_cm
                
          else:
            input_ids_comment= feature_input[0]
            input_mask_comment= feature_input[1]
            input_ids_parent= feature_input[2]
            input_mask_parent= feature_input[3]
            test_label= feature_input[4]
            input_ids_comment = input_ids_comment.to(device)
            input_mask_comment = input_mask_comment.to(device)
            input_ids_parent = input_ids_parent.to(device)
            input_mask_parent = input_mask_parent.to(device)
            test_label=test_label.to(device)
            test_output=torch.sigmoid(torch.squeeze(model(input_ids_comment,input_mask_comment,input_ids_parent,input_mask_parent),1))
            #if input_ids_comment.size()[0] > 4:  # ensures we're looking at sufficiently large comments
            test_output=torch.sigmoid(torch.squeeze(model(input_ids_comment,input_mask_comment,input_ids_parent,input_mask_parent),1))
            test_predicted = torch.round(test_output.data)
            current_cm = confusion_matrix(test_label.cpu().numpy(), test_predicted.cpu().numpy())
            confusion_mtx += current_cm
          #print(test_label.cpu().numpy())
          #print(test_predicted.cpu().numpy())
          #print(current_cm)
          #print(confusion_mtx)
          print(c)
          c=c+1
            
            
    tn, fp, fn, tp = confusion_mtx.ravel()
    accuracy = (tn + tp) / (tn + fp + fn + tp)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    print("Accuracy: ", accuracy)
    print("TPR / Recall / Sensitivity: ", recall)
    print("Precision: ", precision)
    print("F1: ", 2 * (precision * recall) / (precision + recall))
    return tn, fp, fn, tp

# Part 2.  Modeling sarcasm without context

##Question 2.1 Logistic Regression Model

In [0]:
# raise NotImplementedError
# class LogisticRegression(nn.Module):
#     def __init__(self,output_dim):
#         super(LogisticRegression, self).__init__()
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.bert.eval()
#         self.num_labels = output_dim
#         self.classifier = nn.Linear(768, output_dim)

#     def forward(self, input_ids,attention_mask, parent=None):
#         embedded, _ = self.bert(input_ids, None, attention_mask, output_all_encoded_layers=False)
#         embedded = embedded.permute(0, 2, 1)
#         embedded = torch.mean(embedded, dim=2)
#         result=self.classifier(embedded)
# #         print(self.classifier.weight.data)
#         return result

In [0]:
# reset_env()
# # base lr
# # attempt 0: 0.001 // peaks at 7 epochs
# base_lr_hyperparameters = {'learning_rate': 0.01,
#                            'num_epochs': 10} 
# # config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

# logistic_reg = LogisticRegression(all_models_hyperparameters['output_dim']).to(device)
# optimizer = optim.Adam(logistic_reg.parameters(), lr=base_lr_hyperparameters['learning_rate'])
# criterion = nn.BCEWithLogitsLoss().to(device)

# examples=read_examples('./data/reddit_train.csv')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# features = convert_examples_to_features(examples=examples,seq_length=128, tokenizer=tokenizer)
# del examples
# del tokenizer
# gc.collect()
# all_input_ids = torch.tensor([f.input_ids_comment for f in features], dtype=torch.long)
# all_input_mask = torch.tensor([f.input_mask_comment for f in features], dtype=torch.long)
# all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
# eval_data = TensorDataset(all_input_ids, all_input_mask, all_labels)
# eval_sampler = RandomSampler(eval_data)
# eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=all_models_hyperparameters['train_batch_size'])

# logistic_reg = train_model('Logistic-Regression-attempt-0-', logistic_reg, optimizer, criterion, base_lr_hyperparameters['num_epochs'])

# # del all_input_ids
# # del all_input_mask
# # del all_labels
# # del eval_data
# # del eval_sampler
# # del eval_dataloader
# # gc.collect()

In [0]:
# raise NotImplementedError
# # del logistic_reg
# # del optimizer
# # del criterion
# # gc.collect()
# # # del tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# # features = convert_examples_to_features(examples=examples,seq_length=128, tokenizer=tokenizer)

# # all_input_ids = torch.tensor([f.input_ids_comment for f in features], dtype=torch.long)
# # all_input_mask = torch.tensor([f.input_mask_comment for f in features], dtype=torch.long)
# # all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
# # eval_data = TensorDataset(all_input_ids, all_input_mask, all_labels)
# # eval_sampler = RandomSampler(eval_data)
# # eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=all_models_hyperparameters['train_batch_size'])

In [0]:
# logistic_reg = LogisticRegression(all_models_hyperparameters['embedding_dim'], 
#                                   all_models_hyperparameters['output_dim'])
# logistic_reg.load_state_dict(torch.load("/content/gdrive/My Drive/hw-3-models/Logistic-Regression-attempt-0--100000"))

# examples=read_examples('./data/reddit_test.csv')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# features = convert_examples_to_features(examples=examples,seq_length=128, tokenizer=tokenizer)
# del examples
# del tokenizer
# gc.collect()
# all_input_ids = torch.tensor([f.input_ids_comment for f in features], dtype=torch.long)
# all_input_mask = torch.tensor([f.input_mask_comment for f in features], dtype=torch.long)
# all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
# eval_data = TensorDataset(all_input_ids, all_input_mask, all_labels)
# eval_sampler = RandomSampler(eval_data)
# eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=all_models_hyperparameters['test_batch_size'])
# test_model_accuracy(logistic_reg)

In [0]:
# test_model_confusion_matrix(logistic_reg)
# len(eval_data)

# del all_input_ids
# del all_input_mask
# del all_labels
# del eval_data
# del eval_sampler
# del eval_dataloader
# gc.collect()

##Question 2.2 CNN Model

In [0]:
# class CNN1d(nn.Module):
#     def __init__(self, embedding_dim, num_features, filter_sizes, output_dim, dropout):
#         super().__init__()

# #         self.embed = nn.Embedding(len(vocab), embedding_dim)  # we used GLove 100-dim
# #         self.embed.weight.data.copy_(vocab.vectors)
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.bert.eval()

#         self.convs = nn.ModuleList([
#             nn.Conv1d(in_channels=embedding_dim,
#                       out_channels=num_features,
#                       kernel_size=fs)
#             for fs in filter_sizes
#         ])

#         self.fc = nn.Linear(len(filter_sizes) * num_features, output_dim)

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, input_ids,attention_mask, parent=None):
#         embedded, _ = self.bert(input_ids, None, attention_mask, output_all_encoded_layers=False)
#         embedded = embedded.permute(0, 2, 1)
#         conved = [F.relu(conv(embedded)) for conv in self.convs]
#         pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
#         cat = self.dropout(torch.cat(pooled, dim=1))
#         return torch.sigmoid(self.fc(cat))

In [0]:
# # base CNN
# # attempt 0: 60 x [3,4,5] x 0.001, dropout=0.5 (good) 0.7981794259015601
# # attempt 1: 80 x [3,4,5] x 0.0005, dropout=0.2 (bad)
# # attempt 2: 60 x [3,4,5] x 0.001, dropout=0.2 (bad)
# # attempt 3: 80 x [2,3,4,5,6] x  0.0005 x dropout=0.5 (bad)
# # attempt 4: 80 x [1,2,3,4,5] x 0.0005 x 0.5 (good) 0.8047 train; 0.7211 test
# # attempt 5: 80 x [1,2,3,4,5] x 0.0001 x 0.5 dropout x 30 epochs (good) 0.83 train; 0.7022 test
# # attempt 6: 40 x [1,2,3,4,5] x 0.0001 x 0.6 dropout x 15 epochs (good) 0.79 train; 0.707 test
# # attempt 7: 200 x [1, 3, 5] x 0.0005 x 0.5 dropout x 15 epochs (bad)
# # attempt 8: 80 x [1, 1, 2, 2, 3, 3, 4, 5, 8, 10] x 0.0005 x 0.5 dropout x 15 epochs (bad)
# # attempt 9: 200 x [1, 2, 3, 4, 5, 8, 10] x 0.0005 x 0.5 dropout x 15 epochs (bad)
# # attempt 10: 200 x [1, 2, 3] x 0.0001 x 0.5 dropout x 15 epochs (bad)
# # attempt 11: 80 x [1, 2, 3] x 0.0001 x 0.5 dropout x 15 epochs (bad)
# # attempt 12: 20 x [3,4,5] x 0.001 x 0.5 dropout x 10 epochs
# # attempt 13: 40 x [3,4,5] x 0.001 x 0.5 dropout x 20 epochs
# # attempt 14: 20 x [3,4,5] x 0.001 x 0.5 dropout x 20 epochs
# # attempt 14: 20 x [3,4,5] x 0.001 x 0.5 dropout x 25 epochs x bs 40
# base_cnn_hyperparameters = {'num_features': 20,
#                             'filter_sizes': [3, 4, 5], #[2-6] didn't work
#                             'learning_rate': 0.001,
#                             'num_epochs': 25,
#                             'dropout': 0.2}
# cnn = CNN1d(all_models_hyperparameters['embedding_dim'],
#             base_cnn_hyperparameters['num_features'],
#             base_cnn_hyperparameters['filter_sizes'],
#             all_models_hyperparameters['output_dim'],
#             base_cnn_hyperparameters['dropout']).to(device)

# optimizer = optim.Adam(cnn.parameters(), lr=base_cnn_hyperparameters['learning_rate'])

# criterion = nn.BCEWithLogitsLoss().to(device)

# cnn = train_model('CNN-attempt-14-', cnn, optimizer, criterion, base_cnn_hyperparameters['num_epochs'])

In [0]:
# # cnn = CNN1d(all_models_hyperparameters['embedding_dim'],
# #             base_cnn_hyperparameters['num_features'],
# #             base_cnn_hyperparameters['filter_sizes'],
# #             all_models_hyperparameters['output_dim'],
# #             all_models_hyperparameters['dropout']).to(device)
# # cnn.load_state_dict(torch.load("/content/gdrive/My Drive/hw-3-models/CNN-attempt-12--240000"))

# test_model_confusion_matrix(cnn)

##Question 2.3 LSTM Model

In [0]:
# class RNN(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
#                  bidirectional, dropout, pad_idx):
        
#         super().__init__()        
        
#         self.bert = BertModel.from_pretrained('bert-base-uncased')
#         self.bert.eval()
#         self.rnn = nn.LSTM(embedding_dim, 
#                            hidden_dim, 
#                            num_layers=n_layers, 
#                            bidirectional=bidirectional, 
#                            dropout=dropout)
#         self.fc = nn.Linear(hidden_dim * 2, output_dim)
#         self.dropout = nn.Dropout(dropout)
        
#     def forward(self, input_ids,attention_mask,text_lengths, parent=None):
#         embedded, _ = self.bert(input_ids, None, attention_mask, output_all_encoded_layers=False)
        
# #         embedded = self.dropout(self.embedding(text))
        
#         #pack sequence
#         packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
#         packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
#         #unpack sequence
#         output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
#         hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
#         #hidden = [batch size, hid dim * num directions]
            
#         return torch.sigmoid(self.fc(hidden.squeeze(0)))

In [0]:
# # base RNN
# # attempt 0: 15 epochs, 256 hiddens, 2 layers, dropout = 0.2, lr = 0.0005
# # attempt 1: 15 epochs, 128 hiddens, 1 layers, dropout = 0.5, lr = 0.0001 (not quite as good)
# # attempt 2: 15 epochs, 256 hiddens, 1 layers, dropout = 0.2, lr = 0.001
# # attempt 3: 10 epochs, 50 hiddens, 1 layers, dropout = 0.5, lr = 0.001
# # attempt 4: 20 epochs, 100 hiddens, 1 layers, dropout = 0.5, lr = 0.001
# base_rnn_hyperparameters = {'hidden_size': 100,
#                             'number_of_layers': 1,
#                             'bidirectional': True,
#                             'dropout': 0.2,
#                             'pad_idx': TEXT.vocab.stoi[TEXT.pad_token],
#                             'learning_rate': 0.001,
#                             'num_epochs': 20}
# rnn = RNN(all_models_hyperparameters['vocabulary_size'],
#           all_models_hyperparameters['embedding_dim'], 
#           base_rnn_hyperparameters['hidden_size'],
#           all_models_hyperparameters['output_dim'],
#           base_rnn_hyperparameters['number_of_layers'],
#           base_rnn_hyperparameters['bidirectional'],
#           all_models_hyperparameters['dropout'],
#           base_rnn_hyperparameters['pad_idx'])

# optimizer = optim.Adam(rnn.parameters(), lr=base_rnn_hyperparameters['learning_rate'])

# criterion = nn.BCEWithLogitsLoss().to(device)

# rnn = train_model('RNN-attempt-0-', rnn, optimizer, criterion, base_rnn_hyperparameters['num_epochs'], variable_length=True)

In [0]:
# test_model_confusion_matrix(rnn, variable_length=True)

# Part 3.  Modeling sarcasm by concatenating context

## Utility cells for modeling

In [0]:
# ### hyperparameters
# # overall
# all_models_hyperparameters = {'embedding_dim': 100,
#                               'output_dim': 1,
#                               'dropout': 0.5,
#                               'vocabulary_size': len(TEXT.vocab),
#                               'number_of_epochs': 10,
#                               'batch_size': 50}

# # base lr
# # attempt 0: 0.001
# two_utt_lr_hyperparameters = {'learning_rate': 0.001}

# # base CNN
# # attempt 0: 60 x [3,4,5] x 0.001, dropout=0.5 (good) 0.7981794259015601
# # attempt 1: 80 x [3,4,5] x 0.0005, dropout=0.2 (bad)
# # attempt 2: 60 x [3,4,5] x 0.001, dropout=0.2 (bad)
# # attempt 3: 80 x [2,3,4,5,6] x  0.0005 x dropout=0.5 (bad)
# # attempt 4: 80 x [1,2,3,4,5] x 0.0005 x 0.5 (good) 0.8047 train; 0.7211 test
# # attempt 5: 80 x [1,2,3,4,5] x 0.0001 x 0.5 dropout x 30 epochs (good) 0.83 train; 0.7022 test
# # attempt 6: 40 x [1,2,3,4,5] x 0.0001 x 0.6 dropout x 15 epochs (good) 0.79 train; 0.707 test
# # attempt 7: 200 x [1, 3, 5] x 0.0005 x 0.5 dropout x 15 epochs (bad)
# # attempt 8: 80 x [1, 1, 2, 2, 3, 3, 4, 5, 8, 10] x 0.0005 x 0.5 dropout x 15 epochs (bad)
# # attempt 9: 200 x [1, 2, 3, 4, 5, 8, 10] x 0.0005 x 0.5 dropout x 15 epochs (bad)
# # attempt 10: 200 x [1, 2, 3] x 0.0001 x 0.5 dropout x 15 epochs (bad)
# # attempt 11: 80 x [1, 2, 3] x 0.0001 x 0.5 dropout x 15 epochs (bad)
# two_utt_cnn_hyperparameters = {'num_features': 200,
#                             'filter_sizes': [1, 2, 3], #[2-6] didn't work
#                             'learning_rate': 0.0002}

# # base RNN
# # attempt 0: 15 epochs, 256 hiddens, 2 layers, dropout = 0.2, lr = 0.0005
# # attempt 1: 15 epochs, 128 hiddens, 1 layers, dropout = 0.5, lr = 0.0001 (not quite as good)
# # attempt 2: 15 epochs, 256 hiddens, 1 layers, dropout = 0.2, lr = 0.001
two_utt_rnn_hyperparameters = {'hidden_size': 256,
                            'number_of_layers': 1,
                            'bidirectional': True,
                            'dropout': 0.2,
                            'learning_rate': 0.0001}



##Question 3.1 Logistic Regression Model

In [0]:
# class ConcatenatedLogisticRegression(nn.Module):
#     def __init__(self, embedding_dim, output_dim):
#         super().__init__()

#         self.embed = nn.Embedding(len(vocab), embedding_dim)  # we used GLove 100-dim
#         self.embed.weight.data.copy_(vocab.vectors)
#         self.fc = nn.Linear(embedding_dim, output_dim)

#     def forward(self, text, parent=None):
#         text = torch.cat((parent, text))
#         text = text.permute(1, 0)
#         embedded = self.embed(text)
#         embedded = embedded.permute(0, 2, 1)
#         avg_embedded = torch.mean(embedded, dim=2)
#         return torch.sigmoid(self.fc(avg_embedded))

In [0]:
# tick = time.time()
# logistic_reg = ConcatenatedLogisticRegression(all_models_hyperparameters['embedding_dim'], 
#                                   all_models_hyperparameters['output_dim'])

# optimizer = optim.Adam(logistic_reg.parameters(), lr=two_utt_lr_hyperparameters['learning_rate'])

# criterion = nn.BCEWithLogitsLoss().to(device)

# logistic_reg = train_model('cu-Logistic-Regression-attempt-0-', logistic_reg, optimizer, criterion)
# print(time.time() - tick)

In [0]:
# logistic_reg = ConcatenatedLogisticRegression(all_models_hyperparameters['embedding_dim'], 
#                                   all_models_hyperparameters['output_dim'])
# logistic_reg.load_state_dict(torch.load("/content/gdrive/My Drive/cu-Logistic-Regression-attempt-0-"))
# # print("done!")

# test_model_confusion_matrix(logistic_reg, variable_length=False)

##Question 3.2 CNN Model

In [0]:
# class ConcatenatedCNN1d(nn.Module):
#     def __init__(self, embedding_dim, num_features, filter_sizes, output_dim, dropout):
#         super().__init__()

#         self.embed = nn.Embedding(len(vocab), embedding_dim)  # we used GLove 100-dim
#         self.embed.weight.data.copy_(vocab.vectors)

#         self.convs = nn.ModuleList([
#             nn.Conv1d(in_channels=embedding_dim,
#                       out_channels=num_features,
#                       kernel_size=fs)
#             for fs in filter_sizes
#         ])

#         self.fc = nn.Linear(len(filter_sizes) * num_features, output_dim)

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, text, parent=None):
#         text = text.permute(1, 0)
#         parent = parent.permute(1, 0)
#         text = torch.cat((parent, text), dim=1)
#         embedded = self.embed(text)
#         embedded = embedded.permute(0, 2, 1)
#         conved = [F.relu(conv(embedded)) for conv in self.convs]
#         pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
#         cat = self.dropout(torch.cat(pooled, dim=1))
#         return torch.sigmoid(self.fc(cat))

In [0]:
# tick = time.time()
# concat_cnn = ConcatenatedCNN1d(all_models_hyperparameters['embedding_dim'],
#             two_utt_cnn_hyperparameters['num_features'],
#             two_utt_cnn_hyperparameters['filter_sizes'],
#             all_models_hyperparameters['output_dim'],
#             all_models_hyperparameters['dropout']).to(device)

# optimizer = optim.Adam(concat_cnn.parameters(), lr=two_utt_cnn_hyperparameters['learning_rate'])

# criterion = nn.BCEWithLogitsLoss().to(device)

# concat_cnn = train_model('concat-CNN-attempt-0-', concat_cnn, optimizer, criterion)
# print(time.time() - tick)

In [0]:
# concat_cnn = ConcatenatedCNN1d(all_models_hyperparameters['embedding_dim'],
#             base_cnn_hyperparameters['num_features'],
#             base_cnn_hyperparameters['filter_sizes'],
#             all_models_hyperparameters['output_dim'],
#             all_models_hyperparameters['dropout']).to(device)
# concat_cnn.load_state_dict(torch.load("/content/gdrive/My Drive/concat-CNN-attempt-0-"))
# # # print("done!")

# test_model_confusion_matrix(concat_cnn, variable_length=False)

##Question 3.3 LSTM Model

In [0]:
# class ConcatenatedRNN(nn.Module):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
#                  bidirectional, dropout, pad_idx):
        
#         super().__init__()        
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
#         self.rnn = nn.LSTM(embedding_dim, 
#                            hidden_dim, 
#                            num_layers=n_layers, 
#                            bidirectional=bidirectional, 
#                            dropout=dropout)
#         self.fc = nn.Linear(hidden_dim * 2, output_dim)
#         self.dropout = nn.Dropout(dropout)
        
#     def forward(self, text, parent, text_lengths):
        
#         text = torch.cat((parent, text), dim=0)
#         embedded = self.dropout(self.embedding(text))
        
#         #pack sequence
#         packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
#         packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
#         #unpack sequence
#         output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
#         hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
#         #hidden = [batch size, hid dim * num directions]
            
#         return torch.sigmoid(self.fc(hidden.squeeze(0)))

In [0]:
# tick = time.time()
# concat_rnn = ConcatenatedRNN(all_models_hyperparameters['vocabulary_size'],
#           all_models_hyperparameters['embedding_dim'], 
#           two_utt_rnn_hyperparameters['hidden_size'],
#           all_models_hyperparameters['output_dim'],
#           two_utt_rnn_hyperparameters['number_of_layers'],
#           two_utt_rnn_hyperparameters['bidirectional'],
#           all_models_hyperparameters['dropout'],
#           two_utt_rnn_hyperparameters['pad_idx'])

# optimizer = optim.Adam(concat_rnn.parameters(), lr=two_utt_rnn_hyperparameters['learning_rate'])

# criterion = nn.BCEWithLogitsLoss().to(device)

# concat_rnn = train_model('concat-RNN-attempt-0-', concat_rnn, optimizer, criterion, variable_length=True)
# print(time.time() - tick)

In [0]:
# test_model_confusion_matrix(concat_rnn, variable_length=True)

# Part 4.  Modeling sarcasm by separating context

##Utility cells for modeling

In [13]:
### hyperparameters
# overall
all_models_hyperparameters = {'embedding_dim': 768,
                              'output_dim': 1,
                              'dropout': 0.5,
                              'number_of_epochs': 20,
                              'train_batch_size': 20,
                              'test_batch_size': 20}

# # base lr
# # attempt 0: 0.001
separate_lr_hyperparameters = {'learning_rate': 0.1,'num_epochs':1}

# # base CNN
# # attempt 0: 200 x [1,2,3,4,5] x 0.0005 (bad)
# # attempt 1: 200 x [1,2,3,4,5] x 0.0001 (bad)
# # attempt 2: 50 x [3,4,5] x 0.0001 (good)
# # attempt 2.5: 80 x [3,4,5] x 0.00001 (too slow)
# # attempt 3: 80 x [3,4,5] x 0.0005 (good)
# # attempt 3: 80 x [3,4,5] x 0.0005 x 20 epochs (good)
# # jk all these are wrong

# # attempt 5: 20 x [2,3,4] x 0.0005 x 10 epochs (good)
# # attempt 6: 
separate_cnn_hyperparameters = {'num_features': 20,
                            'filter_sizes': [2, 3, 4], #[2-6] didn't work
                            'learning_rate': 0.0005
                               ,'num_epochs':1}

# # base RNN
# # attempt 0: 15 epochs, 256 hiddens, 2 layers, dropout = 0.2, lr = 0.0005
# # attempt 1: 15 epochs, 128 hiddens, 1 layers, dropout = 0.5, lr = 0.0001 (not quite as good)
# # attempt 2: 15 epochs, 256 hiddens, 1 layers, dropout = 0.2, lr = 0.001
separate_rnn_hyperparameters = {'hidden_size': 256,
                            'number_of_layers': 1,
                            'bidirectional': True,
                            'dropout': 0.2,
                            'learning_rate': 0.0001
                               ,'num_epochs':1}
#eval_dataloader=get_features('./data/reddit_train.csv',all_models_hyperparameters['train_batch_size'])
#gc.collect()

100%|██████████| 231508/231508 [00:00<00:00, 5585071.41B/s]


0

In [71]:
test_eval_dataloader=get_features("/content/gdrive/My Drive/subset1.csv",all_models_hyperparameters['train_batch_size'])
gc.collect()

0

In [0]:
#import pandas
#df = pandas.read_csv('./data/reddit_test.csv', 
#            index_col='Unnamed', 
#            parse_dates=['Index'],
#            header=0, 
#            names=['Unnamed','Index','comment','parent_comment'])
#df.to_csv('/content/gdrive/My Drive/test.csv')

In [0]:
#test_eval_dataloader=get_features('./data/reddit_test.csv',all_models_hyperparameters['train_batch_size'])
#gc.collect()

##Question 4.1 Logistic Regression Model

In [0]:
class SeparateLogisticRegression(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        model_state_dict = torch.load('./pytorch_model.bin')
        self.bert = BertModel.from_pretrained('bert-base-uncased',state_dict=model_state_dict)
        self.bert.eval()
        self.num_labels = output_dim
        self.classifier = nn.Linear(2 * 768, output_dim)

    def forward(self, input_ids_comment,attention_mask_comment,input_ids_parent=None,attention_mask_parent=None):
        def mean_embed(input_ids,attention_mask):
            embedded, _ = self.bert(input_ids, None, attention_mask, output_all_encoded_layers=False)
            embedded = embedded.permute(0, 2, 1)
            embedded = torch.mean(embedded, dim=2)
            return embedded
#         print(attention_mask_parent)
        avg_embedded = torch.cat((mean_embed(input_ids_comment,attention_mask_comment), mean_embed(input_ids_parent,attention_mask_parent)), dim=1)
        return self.classifier(avg_embedded)

In [26]:
sep_logistic_reg = SeparateLogisticRegression(all_models_hyperparameters['output_dim']).to(device)
optimizer = optim.Adam(sep_logistic_reg.parameters(), lr=separate_lr_hyperparameters['learning_rate'])
criterion = nn.BCEWithLogitsLoss().to(device)
#sep_logistic_reg = train_model('Sep-Logistic-Regression-attempt-0-', sep_logistic_reg, optimizer, criterion, separate_lr_hyperparameters['num_epochs'],parents=True)


100%|██████████| 407873900/407873900 [00:06<00:00, 62293860.24B/s]


##Question 4.2 CNN Model

In [0]:
class SeparateCNN1d(nn.Module):
    def __init__(self, embedding_dim, num_features, filter_sizes, output_dim, dropout):
        super().__init__()

#         self.embed = nn.Embedding(len(vocab), embedding_dim)  # we used GLove 100-dim
#         self.embed.weight.data.copy_(vocab.vectors)
        model_state_dict = torch.load('./pytorch_model.bin')
        self.bert = BertModel.from_pretrained('bert-base-uncased',state_dict=model_state_dict)
        self.bert.eval()

        self.conv_stack_1 = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim,
                      out_channels=num_features,
                      kernel_size=fs)
            for fs in filter_sizes
        ])
        self.conv_stack_2 = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim,
                      out_channels=num_features,
                      kernel_size=fs)
            for fs in filter_sizes
        ])

        self.classifier = nn.Linear(2 * len(filter_sizes) * num_features, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids_comment,attention_mask_comment,input_ids_parent=None,attention_mask_parent=None):
        def process_utterance(input_ids,attention_mask,stack):
            embedded, _ = self.bert(input_ids, None, attention_mask, output_all_encoded_layers=False)
            embedded = embedded.permute(0, 2, 1)
            conved = [F.relu(conv(embedded)) for conv in stack]
            pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
            cat = self.dropout(torch.cat(pooled, dim=1))
            return cat
        features_from_both = torch.cat((process_utterance(input_ids_parent,attention_mask_parent, self.conv_stack_1),
                                        process_utterance(input_ids_comment,attention_mask_comment, self.conv_stack_2)),
                                       dim=1)
        return self.classifier(features_from_both)

In [0]:
tick = time.time()
separate_cnn = SeparateCNN1d(all_models_hyperparameters['embedding_dim'],
                            separate_cnn_hyperparameters['num_features'],
                            separate_cnn_hyperparameters['filter_sizes'],
                            all_models_hyperparameters['output_dim'],
                            all_models_hyperparameters['dropout']).to(device)

optimizer = optim.Adam(separate_cnn.parameters(), lr=separate_cnn_hyperparameters['learning_rate'])
criterion = nn.BCEWithLogitsLoss().to(device)
#separate_cnn = train_model('separate-CNN-attempt-5-', separate_cnn, optimizer, criterion,separate_lr_hyperparameters['num_epochs'],parents=True)
#print(time.time() - tick)

##Question 4.3 RNN Model

In [0]:
class SeparateRNN(nn.Module):
    def __init__(self,  embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()        
#         self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        model_state_dict = torch.load('./pytorch_model.bin')
        self.bert = BertModel.from_pretrained('bert-base-uncased',state_dict=model_state_dict)
        self.bert.eval()
        self.child_rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.parent_rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        self.classifier = nn.Linear(hidden_dim * 4, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,  input_ids_comment,attention_mask_comment,input_ids_parent=None,attention_mask_parent=None):
        def process_utterance(input_ids,attention_mask,rnn):
            embedded, _ = self.bert(input_ids, None, attention_mask, output_all_encoded_layers=False)
            embedded = embedded.permute(1, 0, 2)

#             #pack sequence
#             packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
#             packed_output, (hidden, cell) = rnn(packed_embedded)

#             #unpack sequence
#             output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
#             hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

            output, (hidden, cell) = rnn(embedded)

            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
            return hidden
        features_from_both = torch.cat((process_utterance(input_ids_parent,attention_mask_parent, self.parent_rnn),
                                        process_utterance(input_ids_comment,attention_mask_comment, self.child_rnn)),
                                       dim=1)
        result= self.classifier(features_from_both)
        return result

In [76]:
tick = time.time()
separate_rnn = SeparateRNN(all_models_hyperparameters['embedding_dim'], 
          separate_rnn_hyperparameters['hidden_size'],
          all_models_hyperparameters['output_dim'],
          separate_rnn_hyperparameters['number_of_layers'],
          separate_rnn_hyperparameters['bidirectional'],
          all_models_hyperparameters['dropout'])

optimizer = optim.Adam(separate_rnn.parameters(), lr=separate_rnn_hyperparameters['learning_rate'])

criterion = nn.BCEWithLogitsLoss().to(device)
#separate_rnn = train_model('separate-RNN-attempt-0-', separate_rnn, optimizer, criterion, separate_rnn_hyperparameters['num_epochs'],parents=True)
#print(time.time() - tick)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


# PART 5. TESTING

In [89]:
## BERT + Logistic
sep_logistic_reg.load_state_dict(torch.load("/content/gdrive/My Drive/hw-3-models/Sep-Logistic-Regression-attempt-0--20000"))
test_model_confusion_matrix(sep_logistic_reg,parents=True)

0
1
2
3
4
5
6
7
8
9
10
Accuracy:  0.44607843137254904
TPR / Recall / Sensitivity:  0.008849557522123894
Precision:  0.5
F1:  0.017391304347826087


(90.0, 1.0, 112.0, 1.0)

In [94]:
## BERT + CNN
separate_cnn.load_state_dict(torch.load("/content/gdrive/My Drive/hw-3-models/separate-CNN-attempt-5--11000"))
test_model_confusion_matrix(separate_cnn,parents=True)

0
1
2
3
4
5
6
7
8
9
10
Accuracy:  0.44607843137254904
TPR / Recall / Sensitivity:  0.008849557522123894
Precision:  0.5
F1:  0.017391304347826087


(90.0, 1.0, 112.0, 1.0)

In [92]:
## BERT + RNN
separate_rnn.load_state_dict(torch.load("/content/gdrive/My Drive/hw-3-models/separate-RNN-attempt-0--16000"))
test_model_confusion_matrix(separate_rnn,parents=True)

0
1
2
3
4
5
6
7
8
9
10
Accuracy:  0.4950980392156863
TPR / Recall / Sensitivity:  0.37168141592920356
Precision:  0.5675675675675675
F1:  0.44919786096256686


(59.0, 32.0, 71.0, 42.0)